# 目的
遺伝子学的分類に基づいた、再発の2値分類を実施する。  

In [1]:
# 基本的なライブラリ
import itertools
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

# models
from sklearn.linear_model import LogisticRegression  # ロジスティック回帰
from sklearn.neighbors import KNeighborsClassifier  # K近傍法
from sklearn.svm import SVC  # サポートベクターマシン
from sklearn.tree import DecisionTreeClassifier, export_graphviz  # 決定木
from sklearn.ensemble import RandomForestClassifier  # ランダムフォレスト
from sklearn.ensemble import AdaBoostClassifier  # AdaBoost
from sklearn.naive_bayes import GaussianNB  # ナイーブ・ベイズ
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA  # 二次判別分析
from sklearn.ensemble import ExtraTreesClassifier
from lightgbm import LGBMClassifier
import xgboost as xgb


# 評価指標
from sklearn.metrics import roc_auc_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


# local libraries
import config
import functions

SEED = config.SEED
functions.fix_seed(SEED)


# 最大表示列数の指定（ここでは50列を指定）N
pd.set_option("display.max_columns", 50)
pd.set_option("display.max_rows", 50)

%matplotlib inline

/home/breast-cancer-analytics/notebooks/functions.py:196: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  y_train: pd.Series(),
/home/breast-cancer-analytics/notebooks/functions.py:198: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  y_val: pd.Series(),


In [2]:
df_recurrenced = pd.read_pickle(
    config.INTERIM_PREPROCESSED_RECURRENCE_DIR + "/df_recurrenced.pkl"
)

classifiers = [
    LogisticRegression(max_iter=2000, random_state=SEED),
    KNeighborsClassifier(),
    SVC(
        kernel="linear",
        random_state=SEED,
        class_weight="balanced",
    ),
    SVC(kernel="poly", random_state=SEED, class_weight="balanced"),
    SVC(kernel="rbf", random_state=SEED, class_weight="balanced"),
    SVC(kernel="sigmoid", random_state=SEED, class_weight="balanced"),
    DecisionTreeClassifier(
        min_samples_split=20,
        min_samples_leaf=15,
        random_state=SEED,
        class_weight="balanced",
    ),
    RandomForestClassifier(
        min_samples_split=20,
        min_samples_leaf=15,
        random_state=SEED,
        class_weight="balanced",
    ),
    AdaBoostClassifier(random_state=SEED),
    GaussianNB(),
    GradientBoostingClassifier(random_state=SEED),
    SGDClassifier(random_state=SEED, class_weight="balanced"),
    QDA(),
    LGBMClassifier(class_weight="balanced", random_state=SEED),
    ExtraTreesClassifier(class_weight="balanced", random_state=SEED),
]

# データ全体での予測モデル検証

## ベースモデルの学習結果

各特徴量選択されたdfを用いて基本的な2値分類モデルを学習する

In [5]:
def validate_models(
    input_file_path: str = None,
    input_file_name: str = None,
    classifiers: list = config.classifiers,
    plot: bool = False,
):
    try:
        list_train, list_val, _, _ = functions.read_preprocessed_df(
            input_file_path, input_file_name
        )
    except:
        return
    X_train, y_train = list_train[0], list_train[1]
    X_val, y_val = list_val[0], list_val[1]
    print(X_train.shape, X_val.shape)
    display(
        functions.compare_bcms(
            X_train,
            y_train,
            X_val,
            y_val,
            classifiers=classifiers,
            plot=plot,
        )
    )


# サブグループ削除（一時的に）
subgroup_columns = [
    "CLAUDIN_SUBTYPE",
    # "NPI_CAT",
    # "TUMOR_CAT",
    # "LYMPH_CAT",
]

feature_selection_methods = [
    # "vt",
    # "mrmr"
    "none"
]

scaling_methods = [
    "std"
    # "norm"
]

dimensionality_reduction_methods = [
    "pca"
    # none
]

sampling_methods = [
    # "none",
    "smote"
]


for (
    feature_selection_method,
    scaling_method,
    dimensionality_reduction_method,
    sampling_method,
) in tqdm(
    itertools.product(
        feature_selection_methods,
        scaling_methods,
        dimensionality_reduction_methods,
        sampling_methods,
    )
):
    preprocess_order = "{0}_{1}_{2}_{3}".format(
        feature_selection_method,
        scaling_method,
        dimensionality_reduction_method,
        sampling_method,
    )
    print("=====" * 10)
    for subgroup_column in subgroup_columns:  # 各サブグループへの適用
        print("-----" * 10)
        print("subgroup column: ", subgroup_column)

        for subgroup in df_recurrenced[subgroup_column].unique():  # サブグループ毎への適用
            print("....." * 10)
            print("subgroup: ", subgroup)
            input_file_path = "./{0}/{1}/{2}".format(
                config.INTERIM_PREPROCESSED_RECURRENCE_DIR,
                subgroup_column,
                preprocess_order,
                plot=True,
            )
            print(input_file_path)
            validate_models(
                input_file_path=input_file_path, input_file_name=subgroup, plot=False
            )

0it [00:00, ?it/s]

--------------------------------------------------
subgroup column:  CLAUDIN_SUBTYPE
..................................................
subgroup:  LumB
./../data/interim/PREPROCESSED/RECURRENCE/CLAUDIN_SUBTYPE/none_std_pca_smote
(216, 158) (20, 158)


,accuracy_score_train,accuracy_score_val,f1_score_train,f1_score_val,matthews_corrcoef_train,matthews_corrcoef_val,cohen_kappa_score_train,cohen_kappa_score_val,roc_auc_score_train,roc_auc_score_val
classifier,,,,,,,,,,
AdaBoostClassifier,1.000000,0.4500,1.000000,0.560000,1.000000,0.057166,1.000000,0.035088,1.000000,0.520833
DecisionTreeClassifier,0.810185,0.6000,0.817778,0.636364,0.622536,0.311805,0.620370,0.259259,0.810185,0.645833
ExtraTreesClassifier,1.000000,0.4000,1.000000,0.571429,1.000000,0.000000,1.000000,0.000000,1.000000,0.500000
GaussianNB,0.810185,0.4000,0.798030,0.571429,0.624914,0.000000,0.620370,0.000000,0.810185,0.500000
GradientBoostingClassifier,1.000000,0.4500,1.000000,0.592593,1.000000,0.187317,1.000000,0.067797,1.000000,0.541667
KNeighborsClassifier,0.541667,0.4000,0.685714,0.571429,0.208514,0.000000,0.083333,0.000000,0.541667,0.500000
LGBMClassifier,1.000000,0.4000,1.000000,0.500000,1.000000,-0.102062,1.000000,-0.071429,1.000000,0.458333
LogisticRegression,1.000000,0.5500,1.000000,0.307692,1.000000,0.000000,1.000000,0.000000,1.000000,0.500000
QuadraticDiscriminantAnalysis,1.000000,0.6000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.500000


..................................................
subgroup:  Her2
./../data/interim/PREPROCESSED/RECURRENCE/CLAUDIN_SUBTYPE/none_std_pca_smote
(134, 80) (11, 80)


,accuracy_score_train,accuracy_score_val,f1_score_train,f1_score_val,matthews_corrcoef_train,matthews_corrcoef_val,cohen_kappa_score_train,cohen_kappa_score_val,roc_auc_score_train,roc_auc_score_val
classifier,,,,,,,,,,
AdaBoostClassifier,1.000000,0.454545,1.000000,0.500000,1.000000,0.288675,1.000000,0.153846,1.000000,0.625000
DecisionTreeClassifier,0.858209,0.454545,0.859259,0.500000,0.716498,0.288675,0.716418,0.153846,0.858209,0.625000
ExtraTreesClassifier,1.000000,0.272727,1.000000,0.200000,1.000000,-0.385758,1.000000,-0.294118,1.000000,0.291667
GaussianNB,0.902985,0.272727,0.897638,0.428571,0.810405,0.000000,0.805970,0.000000,0.902985,0.500000
GradientBoostingClassifier,1.000000,0.727273,1.000000,0.571429,1.000000,0.385758,1.000000,0.377358,1.000000,0.708333
KNeighborsClassifier,0.567164,0.272727,0.694737,0.428571,0.244677,0.000000,0.134328,0.000000,0.567164,0.500000
LGBMClassifier,1.000000,0.727273,1.000000,0.571429,1.000000,0.385758,1.000000,0.377358,1.000000,0.708333
LogisticRegression,1.000000,0.818182,1.000000,0.500000,1.000000,0.516398,1.000000,0.421053,1.000000,0.666667
QuadraticDiscriminantAnalysis,1.000000,0.727273,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.500000


..................................................
subgroup:  LumA
./../data/interim/PREPROCESSED/RECURRENCE/CLAUDIN_SUBTYPE/none_std_pca_smote
(192, 158) (20, 158)


,accuracy_score_train,accuracy_score_val,f1_score_train,f1_score_val,matthews_corrcoef_train,matthews_corrcoef_val,cohen_kappa_score_train,cohen_kappa_score_val,roc_auc_score_train,roc_auc_score_val
classifier,,,,,,,,,,
AdaBoostClassifier,1.000000,0.550,1.000000,0.307692,1.000000,0.301511,1.000000,0.166667,1.000000,0.590909
DecisionTreeClassifier,0.796875,0.500,0.797927,0.166667,0.593782,0.207514,0.593750,0.082569,0.796875,0.545455
ExtraTreesClassifier,1.000000,0.450,1.000000,0.153846,1.000000,-0.033501,1.000000,-0.018519,1.000000,0.489899
GaussianNB,0.901042,0.500,0.904523,0.166667,0.804224,0.207514,0.802083,0.082569,0.901042,0.545455
GradientBoostingClassifier,1.000000,0.500,1.000000,0.166667,1.000000,0.207514,1.000000,0.082569,1.000000,0.545455
KNeighborsClassifier,0.536458,0.500,0.264463,0.166667,0.108335,0.207514,0.072917,0.082569,0.536458,0.545455
LGBMClassifier,1.000000,0.600,1.000000,0.428571,1.000000,0.379980,1.000000,0.252336,1.000000,0.636364
LogisticRegression,1.000000,0.600,1.000000,0.692308,1.000000,0.174078,1.000000,0.157895,1.000000,0.575758
QuadraticDiscriminantAnalysis,1.000000,0.550,1.000000,0.709677,1.000000,0.000000,1.000000,0.000000,1.000000,0.500000


1it [00:09,  9.18s/it]

..................................................
subgroup:  claudin-low
./../data/interim/PREPROCESSED/RECURRENCE/CLAUDIN_SUBTYPE/none_std_pca_smote
pkl file does not exist
..................................................
subgroup:  Basal
./../data/interim/PREPROCESSED/RECURRENCE/CLAUDIN_SUBTYPE/none_std_pca_smote
pkl file does not exist
..................................................
subgroup:  Normal
./../data/interim/PREPROCESSED/RECURRENCE/CLAUDIN_SUBTYPE/none_std_pca_smote
pkl file does not exist
..................................................
subgroup:  NC
./../data/interim/PREPROCESSED/RECURRENCE/CLAUDIN_SUBTYPE/none_std_pca_smote
pkl file does not exist


## xgboost

In [4]:
def validate_models(
    input_file_path: str = None,
    input_file_name: str = None,
    classifiers: list = config.classifiers,
    plot: bool = False,
):
    try:
        list_train, list_val, _, _ = functions.read_preprocessed_df(
            input_file_path, input_file_name
        )
    except:
        return
    X_train, y_train = list_train[0], list_train[1]
    X_val, y_val = list_val[0], list_val[1]
    print(X_train.shape, X_val.shape)
    display(
        functions.compare_bcms(
            X_train,
            y_train,
            X_val,
            y_val,
            classifiers=classifiers,
            plot=plot,
        )
    )


for feature_selection_method, scaling_method, sampling_method in tqdm(
    itertools.product(feature_selection_methods, scaling_methods, sampling_methods)
):
    preprocess_order = "{0}_{1}_{2}".format(
        feature_selection_method, scaling_method, sampling_method
    )
    print("=====" * 10)
    for subgroup_column in subgroup_columns:  # 各サブグループへの適用
        print("-----" * 10)
        print("subgroup column: ", subgroup_column)

        for subgroup in df_recurrenced[subgroup_column].unique():  # サブグループ毎への適用
            print("....." * 10)
            print("subgroup: ", subgroup)
            input_file_path = "./{0}/{1}/{2}".format(
                config.INTERIM_PREPROCESSED_RECURRENCE_DIR,
                subgroup_column,
                preprocess_order,
                plot=True,
            )
            print(input_file_path)

            try:
                list_train, list_val, _, _ = functions.read_preprocessed_df(
                    input_file_path, subgroup
                )
            except:
                pass

            X_train, y_train = list_train[0], list_train[1]
            X_val, y_val = list_val[0], list_val[1]
            print(X_train.shape, X_val.shape)

            xgb_train = xgb.DMatrix(
                X_train, label=y_train, feature_names=X_train.columns
            )
            xgb_val = xgb.DMatrix(X_val, label=y_val, feature_names=X_val.columns)

            param = {
                # 二値分類問題
                "objective": "binary:logistic",
            }
            clf = xgb.train(param, xgb_train)
            y_val_pred_proba = clf.predict(xgb_val)
            y_val_pred = np.where(y_val_pred_proba > 0.5, 1, 0)
            functions.show_scores(y_val, y_val_pred)

0it [00:00, ?it/s]


--------------------------------------------------
subgroup column:  CLAUDIN_SUBTYPE
..................................................
subgroup:  LumB
./../data/interim/PREPROCESSED/RECURRENCE/CLAUDIN_SUBTYPE/none_std_smote
pkl file does not exist


NameError: name 'list_train' is not defined